# 2025 Data-Driven Driver Rankings

The goal of this notebook is to create an objective driver ranking, based on data.

## Methodology

We will use features generated from F1 data and create a normalization algorithm to process each feature category, creating a hypothetical score for each driver in each feature category.

I will explain everything below as I proceed, which should make things much clearer.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

pd.set_option('display.max_columns', None)

In [ ]:
# change path for local imports

if not (Path.cwd() / 'src').exists():
    project_root = Path.cwd().parents[1]
    os.chdir(project_root)
    print(f"Diretório alterado para: {Path.cwd()}")
else:
    print(f"Já estamos na raiz do projeto: {Path.cwd()}")

In [ ]:
from src.analysis.data_viz.plotter import *
from src.analysis.data_viz.constants import TEAM_COLORS, DRIVER_COLORS
from src.analysis.data_viz.constants import JOLPICA_CONSTRUCTOR_RENAME
import notebooks.f1_2025_data_rankings.utils as nb_utils # helpers for this specific notebook
from src.analysis.utils.feature_normalizer import FeatureNormalizer

In [ ]:
GRAFS_DIR = 'notebooks/f1_2025_data_rankings/chart_images/'

# Features and plan for them:

The analysis of the features in this notebook will consider the features to have some amount of validation (see feature_table_validation.ipynb for reference), and in this notebook I will dive deeper in the values produced in each feature for each driver. Explaining and revisiting each one of the features. And than, after, I will try to create a score/power ranking for each one of the drivers.

Also, all of the datasets have received previous data treatment to calculate the features appropriately. Check the extractor for each feature for more details.

The analysis will focus on the 2025 championship.

In [ ]:
# Loading feature datasets:

pace_path = 'data/features/pace_features.csv'
perf_path = 'data/features/performance_features.csv'
exp_path = 'data/features/experience_features.csv'

df_pace = pd.read_csv(pace_path)
df_perf = pd.read_csv(perf_path)
df_exp = pd.read_csv(exp_path)

list_df_features = [df_pace, df_perf, df_exp]

# Data treatment useful for later

def clean_df(df):
    df = nb_utils.filter_year(df, 2025)
    return nb_utils.replace_constructors_names(df)

df_pace, df_perf, df_exp = [clean_df(p) for p in list_df_features]


# Pace features:

In [ ]:
df_pace

In [ ]:
list_pace_features = nb_utils.get_features_column_list(df_pace)
list_pace_features

In [ ]:
selected_feature = 'avg_pace_vs_field'

df_plot = df_pace.sort_values(by=selected_feature, ascending=False)

graf_barras_padrao(
    df_dados=df_plot,
    x_col='driver_surname',
    y_col=selected_feature,
    hue_col='constructor_name',
    cores_map=TEAM_COLORS,
    titulo=f'2025 Championship - Feature Analysis: {selected_feature}',
    ylabel=selected_feature,
    fmt_rotulo='%.2f',
    save_fig=True,
    save_path=GRAFS_DIR,
)

The problem with what I'm showing above is that it is not really the best way to present this kind of value... The differences are really small and it doesn't really show how much the drivers are diverging from each other... Let's treat the features and then we will get back to analyzing them...

# Normalization/Score Creation

I've created a separate module to normalize the features so we can extract more information from visualizing them and then we can later add weights to each one of them and created a combined score for each feature "category".

## How the normalization works:

1- Apply Z-Score normalization to normalize using mean and std. dev., being a better normalization method to not distort the whole series if there are outliers

2- Apply min-max scaling so the numbers are better interpreted -> Here we will have a max value of 10 and a min value of 5 (elite baseline logic, since the analysis is based on comparison and not an actual test score, we are only comparing elite drivers).

**Update after taking a look at the final scores**: Verstappen dominated Tsunoda so much that even with a Z-Score normalization, he was completely skewing the data. I applied the `clip_outliers_sigma` argument to the `robust_normalize` function to limit the influence of these outlier values that, in majority, come from Verstappen, because he has so much advantage in the H2H features.

So, now that I made this quick break to briefly explain the normalization technique. I will apply it to the feature analysis and we can both see the original value and than the normalized value and I will explain the feature as we go.

But before going ahead, I'm going to create a function below to make my life a little easier when analyzing each single feature.

In [ ]:
def create_feature_norm_analysis(df_feature, feature, lower_is_better):

    '''
    Plots chart for feature analysis and returns original dataframe with normalized feature col added
    '''

    df = df_feature.copy()

    df_plot = df.sort_values(by=feature, ascending=False)

    graf_barras_padrao(
        df_dados=df_plot,
        x_col='driver_surname',
        y_col=feature,
        hue_col='constructor_name',
        cores_map=TEAM_COLORS,
        titulo=f'2025 Championship - Feature Analysis: {feature}',
        ylabel=feature,
        xlabel='Driver',
        fmt_rotulo='%.2f',
        show_legend=False,
        save_fig=True,
        save_path=GRAFS_DIR,
    )

    normalizer = FeatureNormalizer()

    df[f'{feature}_norm'] = normalizer.robust_normalize(df[feature], target_range=(5, 10), lower_is_better=lower_is_better, clip_outliers_sigma=1.5).copy()
    df_plot = df.sort_values(by=feature, ascending=False)

    graf_barras_padrao(
        df_dados=df_plot,
        x_col='driver_surname',
        y_col=f'{feature}_norm',
        hue_col='constructor_name',
        cores_map=TEAM_COLORS,
        titulo=f'2025 Championship - Feature Analysis: {feature} Normalized',
        ylabel=f'{feature} Normalized',
        xlabel='Driver',
        fmt_rotulo='%.2f',
        show_legend=False,
        save_fig=True,
        save_path=GRAFS_DIR,
    )

    return df


# Going back to Pace Features

In [ ]:
feature_index = 0

selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

This feature is generated by the following formula:
- For each race, the driver's median pace (pace is represented by the lap time) is calculated;
- The median pace of the field is calculated by getting the median value of all drivers median pace;
- The driver's median pace is then divided by the median pace of the field to get the pace vs field. This calculation is still done in a race level;
- Then, I average the value for the whole year and get the final value for the driver.

So, at the end of the day, the feature in itself is a ratio of the driver's median pace to the median pace of the field. It could be interpreted as a percentage value of the driver's median pace relative to the median pace of the field. With lower values being better, since the pace is represented by the lap time, and lower lap times are better, obviously.

A driver with a pace vs field of 1.0 is the median of the field, while a driver with a pace vs field of 2.0 is twice the median pace of the field, and so on. And for every driver with a pace vs field lower than 1.0, it means that their pace is lower than the median pace of the field.

In [ ]:
selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

This feature is calculated using pretty much the same logic as the previous one, but it compares the driver's median pace to his teammate's median pace instead of the whole field median pace.

We can see that, even though I used the z-score normalization to make outliers not affect the analysis by so much, it will still detect drivers that have a feature value that is far from the mean and give those drivers a high/low score. Just as we can see with Verstappen and Tsunoda on the chart above.

In [ ]:
selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

The feature above is calculated using the following logic:

1. Calculate the standard deviation for every driver in every race stint, than average the value for all the stints in a race to get the standard deviation for each driver in each race;
2. Calculate the mean value for the above to get the standard deviation for each race;
3. Divide the driver's std. dev. by the race std. dev. to get the feature value for each race.
4. Average the value for each driver for the whole season to get the final feature value.

So, like the previous feature, this feature also represents a ration between values, which can also be interpreted as a percentage between the two.

In [ ]:
selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

Again, the formula for the calculation is the same as explained in the previous cell. But now we are comparing the driver's median pace to his teammate's median pace instead of the whole field median pace.

In [ ]:
selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

Here, instead of comparing the standard deviation of the driver's pace to the standard deviation of the field or teammate, we are showing the raw value.

What I found interesting is that Norris shows up as the most consistent driver when looking at the raw value, but when looking at the value for the feature comparing him to his teammate, he has a higher standard deviation.

This can happen because of the way the data is distributed, since we first compare the driver's standard deviation for every race, and than we calculate the mean value for the year. So, a scenario that this phenomena can happen (and probably has), is that Piastri is probably more consistent than Norris in amount of races, but overall, Lando is able to maintain his consistency across races better than Piastri, who could have suffered with a few rounds where he was not as consistent as he should be.


In [ ]:
selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

The feature above is calculated through the following process:

1. Get the driver's best lap time for each qualifying session;
2. Get the best overall time for each qualifying session (usually pole);
3. Divide the driver's best lap time by the best overall time for each qualifying session, getting a ratio of how fast the driver was in comparison to the pole position time;
4. Subtract 1 from the ratio to get a value that represents the "percentage off-pole" of the driver in that session;
5. Average the percentage off-pole for each driver across all qualifying sessions to get the value representative of the whole season.


In [ ]:
selected_feature = list_pace_features[feature_index]

df_pace = create_feature_norm_analysis(df_feature=df_pace, feature=selected_feature, lower_is_better=True)

feature_index += 1

The calculations for this feature is pretty much the same as above but using the teammate's best time as a comparison basis instead of the pole time.

# Performance Features

In [ ]:
df_perf

In [ ]:
list_perf_features = nb_utils.get_features_column_list(df_perf)
list_perf_features

In [ ]:
feature_index = 0

selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Total points in the season, self-explanatory

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=True)

feature_index += 1

Mean finishing position.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=True)

feature_index += 1

Average starting position.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=True)

feature_index += 1

Average difference between the finishing position and the starting position.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Average points obtained in each round.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Team's point share for each driver of the team.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of times a driver outperformed their teammates in qualifying.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of times a driver outperformed their teammates in main race event.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=True)

feature_index += 1

This is the difference between starting position in each race. Is probably highly correlated (if not the same) with other features presented here. Will look into it in a bit.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=True)

feature_index += 1

Average difference between finishing position

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Average difference between points obtained between teammates.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Win rate for quali between teammates.

In [ ]:
selected_feature = list_perf_features[feature_index]

df_perf = create_feature_norm_analysis(df_feature=df_perf, feature=selected_feature, lower_is_better=False)

feature_index += 1

Win rate for main race events between teammates

# Experience Features:

In [ ]:
df_exp

Now we have a few more features created to show the driver experience. Let's take a look:

In [ ]:
list_exp_features = nb_utils.get_features_column_list(df_exp)
list_exp_features

In [ ]:
feature_index = 0

selected_feature = list_exp_features[feature_index]

df_exp = create_feature_norm_analysis(df_feature=df_exp, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of races entered.

In [ ]:
selected_feature = list_exp_features[feature_index]

df_exp = create_feature_norm_analysis(df_feature=df_exp, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of career wins

In [ ]:
selected_feature = list_exp_features[feature_index]

df_exp = create_feature_norm_analysis(df_feature=df_exp, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of career podiums

In [ ]:
selected_feature = list_exp_features[feature_index]

df_exp = create_feature_norm_analysis(df_feature=df_exp, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of career poles.

In [ ]:
selected_feature = list_exp_features[feature_index]

df_exp = create_feature_norm_analysis(df_feature=df_exp, feature=selected_feature, lower_is_better=False)

feature_index += 1

Number of years competed in F1.

# Creating the scores

Ok, now that we have taken a look at all the features, making sure they make sense and also understanding each one of them. I will work on creating a score for each one of them. Using a mapping to create the weight of each feature and I will explain the logic along the way...

## Weights

Before starting the final score for each kind of feature, we need to define the weights for each feature.

I did this in a compoletely subjective way, based on my knowledge of the sport and the features I selected and how I feel each of them should be representative of the final score (for each feature category). I usually attributed more weight to features that compare teammates directly, as I believe this is a better indicator of the driver's performance since, as a premise of this analysis, I consider that drivers in the same team have access to the same resources and equipment.

After calculating the score for each category of feature, I will create a final weight for each one of this final scores, combining them into one final score for each driver. But I will dive deeper into this after presenting the score for each driver in each category.

First let's load the weights for each feature and I will present them below, they range from 1 to 4.

In [ ]:
file_path = 'notebooks/f1_2025_data_rankings/feature_weights.xlsx'

df_weight_pace = pd.read_excel(file_path, sheet_name='pace')
df_weight_perf = pd.read_excel(file_path, sheet_name='perf')
df_weight_exp = pd.read_excel(file_path, sheet_name='exp')

In [ ]:
df_weight_pace

The idea above is to give the *H2H* ones double the weight of the *VS. Pace* ones.

In [ ]:
df_weight_perf

The idea here is to keep the influence of the more "absolute value" features, but not by much, as I think those are merit of the driver but much more a merit of the team.

Apllied more weight to the teammate relative features, as I think those are more merit of the driver, as well as for the *avg_positions_gained_norm* feature because I think this is also a good representation of the driver's hability on track.

In [ ]:
df_weight_exp

And, now, for the experience features, there is really not that much to it, they are pretty self explanatory and the one I'm valuing the most is the number of races. I reduced the weight of the number of wins by a bit because I think this is highly dependent on the team that the driver was/is on.

## Pace Score:

In [ ]:
df_pace

In [ ]:
def create_combined_score(
    df_features: pd.DataFrame,
    feature_weights: pd.DataFrame,
):

    sum_of_weights = feature_weights['weight'].sum()
    list_of_weighted_cols = []

    for feature in feature_weights['column']:
        feature_weight = feature_weights[feature_weights['column'] == feature]['weight'].values[0]
        df_features[f'{feature}_weighted_score'] = df_features[feature] * feature_weight

        # Pego o nome da coluna pra depois ficar fácil de somar todos e obter o total
        list_of_weighted_cols.append(f'{feature}_weighted_score')

    df_features['combined_score'] = df_features[list_of_weighted_cols].sum(axis=1)
    df_features['combined_score'] = df_features['combined_score'] / sum_of_weights

    return df_features

In [ ]:
df_pace_score = create_combined_score(df_pace, df_weight_pace)

In [ ]:
df_pace_score

In [ ]:
graf_barras_padrao(
    df_dados=df_pace_score.sort_values('combined_score'),
    x_col='driver_surname',
    y_col='combined_score',
    cores_map=TEAM_COLORS,
    hue_col='constructor_name',
    titulo='Pace Combined Score',
    ylabel='Score',
    xlabel='Driver',
    fmt_rotulo='%.2f',
    show_legend=False,
    save_fig=True,
    save_path=GRAFS_DIR,
)

As expected if you looked through the feature values, Verstappen takes the lead for this one.

## Performance Score:

In [ ]:
df_perf_score = create_combined_score(df_features=df_perf, feature_weights=df_weight_perf)

In [ ]:
df_perf_score

In [ ]:
graf_barras_padrao(
    df_dados=df_perf_score.sort_values('combined_score'),
    x_col='driver_surname',
    y_col='combined_score',
    cores_map=TEAM_COLORS,
    hue_col='constructor_name',
    titulo='Performance Combined Score',
    ylabel='Score',
    xlabel='Driver',
    fmt_rotulo='%.2f',
    show_legend=False,
    save_fig=True,
    save_path=GRAFS_DIR,
)

## Experience Score:

In [ ]:
df_exp_score = create_combined_score(df_features=df_exp, feature_weights=df_weight_exp)

In [ ]:
df_exp_score

In [ ]:
graf_barras_padrao(
    df_dados=df_exp_score.sort_values('combined_score'),
    x_col='driver_surname',
    y_col='combined_score',
    cores_map=TEAM_COLORS,
    hue_col='constructor_name',
    titulo='Experience Combined Score',
    ylabel='Score',
    xlabel='Driver',
    fmt_rotulo='%.2f',
    show_legend=False,
    save_fig=True,
    save_path=GRAFS_DIR,
)

# Creating final score:

In [ ]:
df_features_dict = {
    'pace': df_pace_score,
    'exp': df_exp_score,
    'perf': df_perf_score
}

def join_combined_scores(df_features_dict):
    id_cols = ['driver_id', 'year', 'driver_full_name', 'driver_surname', 'constructor_name']
    df_final_score = pd.DataFrame()
    for feat_cat, df_feature in df_features_dict.items():
        df_feature = df_feature[id_cols + ['combined_score']].rename(columns={'combined_score': f'combined_score_{feat_cat}'})
        
        if df_final_score.empty:
            df_final_score = df_feature.copy()
        else:
            df_final_score = pd.merge(df_final_score, df_feature, on=id_cols)
    return df_final_score

In [ ]:
df_final_score = join_combined_scores(df_features_dict)
df_final_score

## Weight for each category:

Now that we have all of the combined scores calculated and joined in a single DataFrame, we can assign weights to each category and great one final marvelous score.

In [ ]:
dict_category_weights = {
    'exp':0.2,
    'pace':0.5,
    'perf':0.3
}

In [ ]:
def add_final_score_column(df: pd.DataFrame, dict_category_weights: dict[str, float]) -> pd.DataFrame:
    list_score_cols = ["combined_score_pace", "combined_score_exp", "combined_score_perf"]

    for cat, weight in dict_category_weights.items():
        df[f'combined_score_{cat}_weighted'] = df[f'combined_score_{cat}'] * weight

    list_weighted_cols = [f'{col}_weighted' for col in list_score_cols]

    df['final_score'] = df[list_weighted_cols].sum(axis=1)

    return df

In [ ]:
df_final_score = add_final_score_column(df_final_score, dict_category_weights)

In [ ]:
df_final_score

In [ ]:
graf_barras_padrao(
    df_dados=df_final_score.sort_values('final_score'),
    x_col='driver_surname',
    y_col='final_score',
    cores_map=TEAM_COLORS,
    hue_col='constructor_name',
    titulo='Final Driver Score',
    ylabel='Score',
    xlabel='Driver',
    fmt_rotulo='%.2f',
    show_legend=False
)

# Radar Chart for Drivers:

In [ ]:
def plot_driver_radar_chart(
    df_scores: pd.DataFrame,
    driver_name: str,
    save_flag: bool = False,
):

    df_driver = df_scores[df_scores['driver_full_name'] == driver_name].rename(
        columns={
            'combined_score_pace':'Pace',
            'combined_score_exp': 'Exp.',
            'combined_score_perf': 'Perf.',
        }
    )[
        [
            'driver_full_name',
            'driver_surname',
            'constructor_name',
            'Pace',
            'Exp.',
            'Perf.',
            'final_score'
        ]
    ].copy(

    )

    dict_infos = df_driver[
        [
            'Pace',
            'Exp.',
            'Perf.',
        ]
    ].to_dict(orient='records')[0]
    cor_base = TEAM_COLORS[df_driver['constructor_name'].values[0]]
    final_score = df_driver['final_score'].values[0]

    graf_radar_padrao(
        dados=dict_infos,
        titulo=f"Driver Score - {driver_name}",
        cor_base=cor_base,
        center_value=final_score,
        tip_value_fmt="{:.2f}",
        center_fontsize=50,
        save_fig=save_flag,
        save_path=f"notebooks/f1_2025_data_rankings/chart_images/",
        tip_fontsize=18,
        max_val=12,
        center_value_fmt="{:.2f}",
    )


In [ ]:
for driver in df_final_score.sort_values(by=['constructor_name'])['driver_full_name'].unique():
    plot_driver_radar_chart(
        df_scores=df_final_score, 
        driver_name=driver, 
        save_flag=True
    )

# Comparing drivers:

Here I will create a functionality to compare 2 drivers using all of the features in a given dataset. Will be useful to compare drivers and understand the score difference.

In [ ]:
df_pace

In [ ]:
def create_unpivoted_comparison_df(
    df_features: pd.DataFrame,
    feature_list: List,
    drivers_to_compare: List,
    norm_features: bool = False
) -> pd.DataFrame:

    df_drivers = df_features[df_features['driver_full_name'].isin(drivers_to_compare)]

    if norm_features:
        feature_list = [feature + '_norm' for feature in feature_list]

    df_compare = df_drivers[['driver_surname', 'driver_full_name'] + feature_list] 

    df_compare = df_compare.melt(
        id_vars=['driver_surname', 'driver_full_name'],
        value_vars=feature_list,
        var_name='feature',
        value_name='feature_value'
    )

    return df_compare


In [ ]:
df_compara_ocon_bearman = create_unpivoted_comparison_df(
    df_pace,
    list_pace_features,
    drivers_to_compare=['Oliver Bearman', 'Esteban Ocon'],
    norm_features=True
)

In [ ]:
def plot_comparison_df(
    df_compare: pd.DataFrame
):

    drivers_in_comparison = df_compare['driver_surname'].unique()

    graf_barras_padrao(
        df_dados=df_compare,
        x_col='feature',
        y_col='feature_value',
        hue_col='driver_surname',
        titulo='Feature Comparison for ' + ', '.join(drivers_in_comparison),
        cores_map=DRIVER_COLORS,
        save_fig=False,
        dodge=True,
        fmt_rotulo='%.2f'
    )

In [ ]:
plot_comparison_df(df_compara_ocon_bearman)

I was curious to see why Bearman has such a low Pace score, and it's mainly beacuse of the consistency scores.